In [22]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans

In [23]:
#Link of wikipedia Pages for Toronto Postal codes
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webpage = requests.get(URL)

In [24]:
#Covert page to BeautifulSoup Object and extract the table
soup = BeautifulSoup(webpage.content, 'html5lib') 
table = soup.find('tbody')

In [25]:
#Load table data to a dataframe
postalArr = []
boroughArr = []
hoodArr = []

for ind, row in enumerate(table.findAll('tr')):
    if (ind == 0):
        continue
    else:
        postalArr.append(row.findChildren()[0].text[:-1])
        boroughArr.append(row.findChildren()[1].text[:-1])
        hoodArr.append(row.findChildren()[2].text[:-1])

torontoDict = {'Postal_Code':postalArr, 'Borough':boroughArr, 'Neighborhood':hoodArr}
df = pd.DataFrame(torontoDict)

#Dataframe from wikipedia Table is created

In [26]:
#Clean the dataframe
df = df[df['Borough'] != 'Not assigned']
df = df.sort_values('Postal_Code').reset_index(drop=True)
display(df)
print('Shape of dataframe is : {}'.format(df.shape))

#Dataframe is cleaned and sorted

,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Shape of dataframe is : (103, 3)


In [27]:
#This dataframe splits all the Neighborhood

df_new = pd.DataFrame(columns=['Postal_Code', 'Borough', 'Neighborhood'])

for i in range(len(df)):
    postal_code = df.loc[i,'Postal_Code']
    borough = df.loc[i,'Borough']
    hoodlist = df.loc[i,'Neighborhood']
    
    for j in hoodlist.split('/'):
        df_new = df_new.append({'Postal_Code': postal_code, 'Borough': borough, 'Neighborhood': j.strip()}, ignore_index=True)

#Dataframe df_new is created

In [28]:

def getLocation(postal_code):
    location = None
    latitude = None
    longitude = None
    
    address = postal_code+', Toronto, Ontario'
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    if (location != None):
        latitude = location.latitude
        longitude = location.longitude
    
    return latitude, longitude

In [29]:
#Load Latitude and Longitude Values from online CSV file

df_ll = pd.read_csv('https://cocl.us/Geospatial_data')
df_ll.rename(columns={'Postal Code':'Postal_Code'}, inplace = True)
df_ll = df_ll.sort_values('Postal_Code').reset_index(drop=True)

#Data loaded to dataframe df_ll

In [30]:

#Combine the two dataframes
df2 = pd.merge(df, df_ll, how ='inner', on ='Postal_Code')
df2.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [31]:
#Get Latitude and longitude values of Toronto
torontoStr = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="t_explorer")
locationToronto = geolocator.geocode(torontoStr)
locationToronto = [locationToronto.latitude, locationToronto.longitude]
print(locationToronto)

[43.6534817, -79.3839347]


In [32]:
#Create map of toronto and mark the postal codes along with 500m radius around them

map_Toronto = folium.Map(location= locationToronto, zoom_start=10)

for lat, long in zip(df2['Latitude'], df2['Longitude']):
   
    folium.Circle([lat, long],
                  radius=1000,
                  color = '#ffd633',
                  opacity = 0.2,
                  fill_color = '#ffd633',
                  fill_opacity = 0.2,
                   ).add_to(map_Toronto)
    
for pc, bor, neigh, lat, long in zip(df2['Postal_Code'], df2['Borough'], df2['Neighborhood'], df2['Latitude'], df2['Longitude']):
    label = '{}, {}, {}'.format(pc, bor, neigh)
    
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color = '#ff3333',
        fill_color = '#ff3333',
        fill_opacity = 1,
        parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [33]:
#Credentials to access Foursquare API

In [34]:
CLIENT_ID = '****************************************************' # your Foursquare ID
CLIENT_SECRET = '****************************************************' # your Foursquare Secret
VERSION = '******' # Foursquare API version


In [35]:
#Function to get URL for each postal code (explore)

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # Radius of Search

def getURL(lat, long):

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
    return url

In [36]:

#Create a new dataframe to store the venue data along with the previous data
df3Columns = list(df2.columns)
addColumns = ['Venue', 'VLatitude', 'VLongitude', 'Category']
df3Columns.extend(addColumns)

df3 = pd.DataFrame(columns = df3Columns)

#Loop over each Postal Code
for row in range(len(df2)):
    pc = df2.loc[row, 'Postal_Code']
    bor = df2.loc[row, 'Borough']
    neigh = df2.loc[row, 'Neighborhood']
    lat = df2.loc[row, 'Latitude']
    long = df2.loc[row, 'Longitude']
    
    try:
        url = getURL(lat,long)
        results = requests.get(url).json()
        itemsDict = results['response']['groups'][0]['items']
    except:
        continue
    
    #Loop over every venue in each postal code
    for item in itemsDict:
        try:
            venueName = item['venue']['name']
            vLat = float(item['venue']['location']['lat'])
            vLong = float(item['venue']['location']['lng'])
            category = item['venue']['categories'][0]['name']

            newRow = [pc, bor, neigh, lat, long, venueName, vLat, vLong, category]
            
            #Append data to the new dataFrame
            df3 = df3.append(pd.Series(newRow, index = df3Columns), ignore_index = True)
        except:
            continue

In [37]:
display(df3)
print('Shape of the dataframe is : {}'.format(df3.shape))

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Venue,VLatitude,VLongitude,Category
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Shape of the dataframe is : (2136, 9)


In [38]:
#Display all venue categories
total_categories = len(df3['Category'].unique())
print('Total Venue categories are : {}'.format(total_categories))

Total Venue categories are : 273


In [39]:

#Create new dataframe of the postal codes and the venue categories
torontoVenues = pd.get_dummies(df3[['Category']], prefix="", prefix_sep="")
torontoVenues.insert(0, 'Postal_Code', df3[['Postal_Code']])
torontoVenues = torontoVenues.groupby('Postal_Code').mean().reset_index()

In [40]:
display(torontoVenues.head())
print('The shape of this dataframe is {}'.format(torontoVenues.shape))

,Postal_Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.

The shape of this dataframe is (100, 274)


In [41]:
#Model to create clusters

kclusters = 6

df_clustering = torontoVenues.drop('Postal_Code',axis = 1)

kMeansModel = KMeans(n_clusters = kclusters, random_state = 0)
kMeansModel.fit(df_clustering)

clustered_arr = kMeansModel.labels_

In [42]:
#Final dataframe showing which postal code belongs to which cluster

final_df = df3[['Postal_Code', 'Latitude', 'Longitude']].groupby('Postal_Code').mean().reset_index()
final_df['Clusters'] = clustered_arr
display(final_df.head())

,Postal_Code,Latitude,Longitude,Clusters
0,M1B,43.806686,-79.194353,4
1,M1C,43.784535,-79.160497,1
2,M1E,43.763573,-79.188711,0
3,M1G,43.770992,-79.216917,0
4,M1H,43.773136,-79.239476,0


In [43]:
#Show the different clusters on map

clusters_map = folium.Map(location = locationToronto, zoom_start = 10)
colors = ['#ff4d4d', '#4da6ff', '#5cd65c', '#ff66ff', '#ffc34d', '#333333']

for pc, lat, long, cluster in zip(final_df['Postal_Code'], final_df['Latitude'], final_df['Longitude'], final_df['Clusters']):
    label = 'Cluster:{}, {}'.format(cluster, pc)
    
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color = colors[cluster],
        fill_color = colors[cluster],
        fill_opacity = 1,
        parse_html=False).add_to(clusters_map)
    
clusters_map

In [44]:
#The number of rows in final dataframe
print('The number of rows in final dataframe is {}'.format(final_df.shape[0]))

The number of rows in final dataframe is 100
